There have been many reports of fake videos of popular celebrities or politicians. These manipulated videos are created by manipulating original videos. These fake videos are difficult to detect with the naked eye and are becoming a major problem in society.

It has been experienced that the Deepfake videos go easily viral on Social media platforms like **Facebook**, **twitter**, **youtube**, etc. They are the main distribution channel for these manipulated videos. While these platforms are working to solve this problem, Facebook is making a large investment ($ 10 million) to address this problem and other platforms like Twitter and Google are also working to solve this problem. These are some [more details](https://ai.googleblog.com/2019/09/contributing-data-to-deepfake-detection.html) on Google's fight against deep fakes.

Deepfake detection is therefore not an easy task. In this project, we will see how to identify the fakes from the real ones. It includes breaking videos into a frame, face detection from real and fake videos, face cropping and analysis. We will use:

1. `OpenCV`: OpenCV supports many algorithms related to computer vision and machine learning. OpenCV has a python library called **OpenCV-Python**.
2. `Numpy`: is used for general representation and manipulation of arrays in Python.
3. `scikit-image`: We will use this tool for face image operations.
4. `Matplotlib`: is used to plot the survey results (histograms andROC curves, etc.).

We are using [the DeepfakeTIMIT database](https://www.idiap.ch/dataset/deepfaketimit) , a video database where faces are swapped using the open source GAN based approach.

This dataset was created [by the Deepfake algorithm based on the original autoencoder](https://github.com/deepfakes/faceswap).

We will train / test using two different trained GAN models: a lower quality (LQ) with a `64 x 64` input / output size model and a higher quality (HQ) with a `128 x 128` size model.

Let's move on to the main topic.

### OpenCV

OpenCV installation using pip

`pip install opencv`

Please follow [this](https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_setup/py_setup_in_windows/py_setup_in_windows.html) article:

OpenCV will be used for four main activities.

* Image processing
* Video processing
* Feature detection
* Object detection

To use OpenCV, all we need to do is

`import cv2`

`# Load an color image in grayscale
img = cv2.imread('imagename.jpg',0)`

* `cv2.IMREAD_COLOR`: Upload a color image. Any transparency of the image will be neglected. It is the default flag.
* `cv2.IMREAD_GRAYSCALE`: Loads the image in grayscale mode
* `cv2.IMREAD_UNCHANGED`: Load image as such, including alpha channel

### View an image

`cv2.imshow('image',img)
 cv2.waitKey(0)
 cv2.destroyAllWindows()`
 
### Write an image

`cv2.imwrite('imagename.png',img)`

### Capture video from the camera

`import numpy as np
 import cv2
 cap = cv2.VideoCapture(0)`

`while(True):
    ret, frame = cap.read() # Capture frame-by-frame
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # Our operations on the frame come here
    cv2.imshow('frame',gray) # Display the resulting frame
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break`

`# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()`

### This will open a video from the camera/Playing a video from a file.

`import numpy as np
import cv2
cap = cv2.VideoCapture('yourvoideofilename.avi')`

`while(cap.isOpened()):
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    cv2.imshow('frame',gray)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break`

`cap.release()
cv2.destroyAllWindows()`

A video file from the path provided will open. These are the general tasks performed in OpenCV, but we can find a lot more [here](https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_gui/py_drawing_functions/py_drawing_functions.html) .

### Matplotlib

Matplotlib will provide us with a wide variety of printing methods. Here is an example of how to display the image using matplotlib.

`import numpy as np
import cv2
from matplotlib import pyplot as plt
img = cv2.imread('yourimagename.jpg',0)
plt.imshow(img, cmap = 'gray', interpolation = 'bicubic')
plt.xticks([]), plt.yticks([]) # to hide tick values on X and Y axis
plt.show()`

Now the main goal is to read a video file module path and extract frames from real videos and fake videos of our choice. Then perform face detection to extract faces. Next, we're analyzing the differences in these two face images using various measurements. We are dividing the codes into several sections.

In [1]:
# !pip install mtcnn

In [2]:
#required Libraries
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pandas as pd
import glob2
import os, fnmatch
from pathlib import Path
# import mtcnn
from mtcnn.mtcnn import MTCNN # from mtcnn import MTCNN

Let’s say we have two types of videos, one real and one fake. We want to detect which is the fake one amongst the two. Probably this is what we need to do in the task of Deepfake detection.

Now, we will create a function to process both the videos

In [6]:
# reading video fame
# Create a VideoCapture object and read from input file

def extract_multiple_videos(intput_filenames, image_path_infile):
    
    """Extract video files into sequence of images."""
    
    i = 1  # Counter of first video
    
    # Iterate file names:
    cap = cv2.VideoCapture('your_video_file_path.avi' or intput_filenames)
    
    if (cap.isOpened()== False):
            print("Error opening file")
    
    # Keep iterating break
    while True:
        ret, frame = cap.read()  # Read frame from first video

        if ret:
            cv2.imwrite(os.path.join(image_path_infile , str(i) + '.jpg'), frame)  # Write frame to JPEG file (1.jpg, 2.jpg, ...)
        #  cv2.imshow('frame', frame)  # Display frame for testing. We can uncomment this line if we want to view them.
            i += 1 # Advance file counter
        else:
            # Break the interal loop when res status is False.
            break
    
    cv2.waitKey(50) # Wait 50msec
    cap.release()

In [10]:
# extract_multiple_videos(fake_video_name, fake_image_path_for_frame)
# extract_multiple_videos(real_video_name, real_image_path_for_frame)

After running the function we just uploaded the video, read the frame and wrote to a file.

Now we find the difference between the two frames and visualize the differences.

In [7]:
from skimage import measure

def mse(imageA, imageB):
    # the 'Mean Squared Error' between the two images is the
    # sum of the squared difference between the two images;
    # NOTE: the two images must have the same dimension
    err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
    err /= float(imageA.shape[0] * imageA.shape[1])
    # return the MSE, the lower the error, the more "similar"
    # the two images are
    return err

In [8]:
def compare_images(imageA, imageB, title):
    # compute the mean squared error and structural similarity
    # index for the images
    m = mse(imageA, imageB)
    s = measure.compare_ssim(imageA, imageB)
    # setup the figure
    fig = plt.figure(title)
    plt.suptitle("MSE: %.2f, SSIM: %.2f" % (m, s))
    # show first image
    ax = fig.add_subplot(1, 2, 1)
    plt.imshow(imageA, cmap = plt.cm.gray)
    plt.axis("off")
    # show the second image
    ax = fig.add_subplot(1, 2, 2)
    plt.imshow(imageB, cmap = plt.cm.gray)
    plt.axis("off")
    # show the images
    plt.show()

In the code above, we are comparing the extracted images from the original video and the corresponding image from fake videos. In the last section of the code, we checked if both the two images have any differences.

In [12]:
original = cv2.imread("image_path_from_real_video")
shopped = cv2.imread("image_path_from_fake_video")

# convert the images to grayscale
original = cv2.cvtColor(original, cv2.COLOR_BGR2GRAY)
shopped = cv2.cvtColor(shopped, cv2.COLOR_BGR2GRAY)

In [ ]:
# initialize the figure
fig = plt.figure("Images")
images = ("Original", original), ("modified", shopped)

# loop over the images
for (i, (name, image)) in enumerate(images):
    # show the image
    ax = fig.add_subplot(1, 3, i + 1)
    ax.set_title(name)
    plt.imshow(image, cmap = plt.cm.gray)
    plt.axis("off")
    # show the figure
    plt.show()

In [ ]:
# compare the images
compare_images(original, original, "Original vs. Original")
compare_images(original, shopped, "Original vs. Modified")
# cv2.subtract(original)
# img3 = original-shopped
image3 = cv2.absdiff(original, shopped)
image3

In [ ]:
is_all_zero = not np.any(image3)

if is_all_zero:
    print('Array contains only 0')
else:
    print('Array has non-zero items too')

We are now irritated that those two images are different. We can calculate **SSIM**.

In [ ]:
# Calculate the structural similarity index (SSIM)

from skimage.measure import compare_ssim
import argparse
import imutils
(score, diff) = compare_ssim(original, shopped, full=True)
diff = (diff * 255).astype(“uint8”)
print(“SSIM: {}”.format(score))

In [ ]:
# Find the differences in the pictures

thresh = cv2.threshold(diff, 0, 255,cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)

In [ ]:
%matplotlib inline# loop over the contours
for c in cnts:
    # compute the bounding box of the contour and then draw the
    # bounding box on both input images to represent where the two
    # images differ
    (x, y, w, h) = cv2.boundingRect(c)
    cv2.rectangle(original, (x, y), (x + w, y + h), (0, 0, 255), 2)
    cv2.rectangle(shopped, (x, y), (x + w, y + h), (0, 0, 255), 2)

# show the output images
# plt.imshow(“Original”, original)
plt.imshow(original)
# cv2.waitKey(0)

# Modified image
# plt.imshow(“Modified”, shopped)
plt.imshow(shopped)

#Differences in two images
# plt.imshow(“diff”, diff)
plt.imshow(diff)

From the images, we can see what the differences were between the two images and calculate the modified part.

In [ ]:
# Perform face detection

from matplotlib.patches import Rectangle
from matplotlib.patches import Circle
from matplotlib import pyplot
from mtcnn.mtcnn import MTCNN

# draw an image with detected objects
def draw_image_with_boxes(filename, result_list, face_filename):
    
    # load the image
    data = pyplot.imread(filename)
    
    for i in range(len(result_list)):
        # get coordinates
        x1, y1, width, height = result_list[i][‘box’]
        x2, y2 = x1 + width, y1 + height
        # define subplot
        pyplot.subplot(1, len(result_list), i+1)
        pyplot.axis(‘off’)
        # plot face
        pyplot.imshow(data[y1:y2, x1:x2])
        pyplot.savefig(‘Dataset/only_face/’+ face_filename)

    # show the plot
    pyplot.show()

In [ ]:
# reading video fame
# Create a VideoCapture object and read from input file

def extract_multiple_videos_faces(intput_video_file_names, image_path_infile):
    """Extract video files into sequence of images.
       Intput_filenames is a list for video file names"""
    i = 1  # Counter of first video
    
    # Iterate file names:
    cap = cv2.VideoCapture('/Users/praladneupane/Documents/Third_Semester/695/projects/Deepfake/Dataset/VidTIMIT/mstk0/sa1.avi')
        
    if (cap.isOpened()== False):
        print("Error opening video stream or file")
    
    # Keep iterating break
    while True:
        ret, frame = cap.read()  # Read frame from first video
            
        if ret:
            
#           cv2.imwrite(str(i) + '.jpg', frame)  # Write frame to JPEG file (1.jpg, 2.jpg, ...)
#             this code can be use to do 1.2.3 and 1.3, but i am only doing 1.3 now
#             cv2.imwrite(os.path.join(image_path_infile , str(i) + '.jpg'), frame)  # Write frame to JPEG file (1.jpg, 2.jpg, ...)
                
#           cv2.imshow('frame', frame)  # Display frame for testing
    
            filename = os.path.join(image_path_infile , str(i) + '.jpg') 
        
        # this line feels little odd. Cause it looks 
        # like i am reading it mannually but if i uncomment above line it will be dynamic everytime, cause those line 
        # creates the frame in the folder first.
        
            # load image from file
            pixels = pyplot.imread(filename)
            # create the detector, using default weights
            detector = MTCNN()
            # detect faces in the image
            faces = detector.detect_faces(pixels)
            # display faces on the original image
            face_filename_crp = str(i) + '.jpg'
            draw_image_with_boxes(filename, faces, face_filename_crp)
        
    
            i += 1 # Advance file counter
        
            face_filename = str(i) + '.jpg'
            # display faces on the original image
            # draw_image_with_boxes(filename, faces,face_filename)
        else:
            # Break the interal loop when res status is False.
            break
    cv2.waitKey(100) #Wait 100msec (for debugging)
    cap.release() #Release must be inside the outer loop

In [ ]:
extract_multiple_videos_faces(video_file_path,real_image_path_for_frame)

We now have the unique face images both real and fake. We can try to implement the above section called **Find Outlines** . It was intentional, so give it a try.

In conclusion, we are able to extract frames from real and fake videos, compare and contrast the same image from both and detect the differences.